In [21]:
!pip install roboflow

from roboflow import Roboflow
import os

rf = Roboflow(api_key="W8Wh3vwPre13GJ9ArQue")
project = rf.workspace("brad-dwyer").project("car-parts-pgo19")
version = project.version(6)
dataset = version.download("sam2")

# rename dataset.location to "data"
os.rename(dataset.location, "/home/iabd/Escritorio/IABD/IA/SAPA/Redes Neuronales/ej10Sam/data")

loading Roboflow workspace...
loading Roboflow project...


In [1]:
!git clone https://github.com/facebookresearch/sam2.git

Clonando en 'sam2'...
remote: Enumerating objects: 1070, done.
remote: Total 1070 (delta 0), reused 0 (delta 0), pack-reused 1070 (from 1)
Recibiendo objetos: 100% (1070/1070), 134.70 MiB | 15.23 MiB/s, listo.
Resolviendo deltas: 100% (375/375), listo.


In [2]:
!wget -O /home/iabd/Escritorio/IABD/IA/SAPA/Redes Neuronales/ej10Sam/sam2/configs/train.yaml 'https://drive.usercontent.google.com/download?id=11cmbxPPsYqFyWq87tmLgBAQ6OZgEhPG3'
%cd ./sam2/
!pip install -e .[dev] -q
!cd ./checkpoints && ./download_ckpts.sh

--2025-02-05 18:39:22--  http://neuronales/ej10Sam/sam2/configs/train.yaml
Resolviendo neuronales (neuronales)... falló: Fallo temporal en la resolución del nombre.
wget: no se pudo resolver la dirección del equipo ‘neuronales’
--2025-02-05 18:39:22--  https://drive.usercontent.google.com/download?id=11cmbxPPsYqFyWq87tmLgBAQ6OZgEhPG3
Resolviendo drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.184.1, 2a00:1450:4003:80c::2001
Conectando con drive.usercontent.google.com (drive.usercontent.google.com)[142.250.184.1]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 11055 (11K) [application/octet-stream]
Guardando como: ‘/home/iabd/Escritorio/IABD/IA/SAPA/Redes’

/home/iabd/Escritor 100%[===================>]  10,80K  --.-KB/s    en 0,002s  

2025-02-05 18:39:24 (4,92 MB/s) - ‘/home/iabd/Escritorio/IABD/IA/SAPA/Redes’ guardado [11055/11055]

ACABADO --2025-02-05 18:39:24--
Tiempo total de reloj: 2,4s
Descargados: 1 ficheros, 11K en 0,002

In [3]:
# Script to rename roboflow filenames to something SAM 2.1 compatible.
# Maybe it is possible to remove this step tweaking sam2/sam2/configs/train.yaml.
import os
import re

FOLDER = "/home/iabd/Escritorio/IABD/IA/SAPA/Redes Neuronales/ej10Sam/data/train"

for filename in os.listdir(FOLDER):
    # Replace all except last dot with underscore
    new_filename = filename.replace(".", "_", filename.count(".") - 1)
    if not re.search(r"_\d+\.\w+$", new_filename):
        # Add an int to the end of base name
        new_filename = new_filename.replace(".", "_1.")
    os.rename(os.path.join(FOLDER, filename), os.path.join(FOLDER, new_filename))

In [4]:
!python training/train.py -c 'configs/train.yaml' --use-cluster 0 --num-gpus 1

Traceback (most recent call last):
  File "/home/iabd/Escritorio/IABD/IA/SAPA/Redes Neuronales/ej10Sam/sam2/training/train.py", line 270, in <module>
    main(args)
  File "/home/iabd/Escritorio/IABD/IA/SAPA/Redes Neuronales/ej10Sam/sam2/training/train.py", line 124, in main
    cfg = compose(config_name=args.config)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iabd/anaconda3/envs/CloneSAPA/lib/python3.12/site-packages/hydra/compose.py", line 38, in compose
    cfg = gh.hydra.compose_config(
          ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iabd/anaconda3/envs/CloneSAPA/lib/python3.12/site-packages/hydra/_internal/hydra.py", line 594, in compose_config
    cfg = self.config_loader.load_configuration(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iabd/anaconda3/envs/CloneSAPA/lib/python3.12/site-packages/hydra/_internal/config_loader_impl.py", line 142, in load_configuration
    return self._load_configuration_impl(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 

In [5]:
%load_ext tensorboard


In [7]:
import torch
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
import supervision as sv
import os
import random
from PIL import Image
import numpy as np

# use bfloat16 for the entire notebook
# from Meta notebook
torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
if torch.cuda.get_device_properties(0).major >= 8:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

checkpoint = "/home/iabd/Escritorio/IABD/IA/SAPA/Redes Neuronales/ej10Sam/sam2/sam2_logs/configs/train.yaml/checkpoints/checkpoint.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_b+.yaml"
sam2 = build_sam2(model_cfg, checkpoint, device="cuda")
mask_generator = SAM2AutomaticMaskGenerator(sam2)

checkpoint_base = "/content/sam2/checkpoints/sam2.1_hiera_base_plus.pt"
model_cfg_base = "configs/sam2.1/sam2.1_hiera_b+.yaml"
sam2_base = build_sam2(model_cfg_base, checkpoint_base, device="cuda")
mask_generator_base = SAM2AutomaticMaskGenerator(sam2_base)

FileNotFoundError: [Errno 2] No such file or directory: '/home/iabd/Escritorio/IABD/IA/SAPA/Redes Neuronales/ej10Sam/sam2/sam2_logs/configs/train.yaml/checkpoints/checkpoint.pt'

In [ ]:
validation_set = os.listdir("/content/data/valid")

# choose random with .json extension
image = random.choice([img for img in validation_set if img.endswith(".jpg")])
image = os.path.join("/content/data/valid", image)
opened_image = np.array(Image.open(image).convert("RGB"))
result = mask_generator.generate(opened_image)

detections = sv.Detections.from_sam(sam_result=result)

mask_annotator = sv.MaskAnnotator(color_lookup = sv.ColorLookup.INDEX)
annotated_image = opened_image.copy()
annotated_image = mask_annotator.annotate(annotated_image, detections=detections)

base_annotator = sv.MaskAnnotator(color_lookup = sv.ColorLookup.INDEX)

base_result = mask_generator_base.generate(opened_image)
base_detections = sv.Detections.from_sam(sam_result=base_result)
base_annotated_image = opened_image.copy()
base_annotated_image = base_annotator.annotate(base_annotated_image, detections=base_detections)

sv.plot_images_grid(images=[annotated_image, base_annotated_image], titles=["Fine-Tuned SAM-2.1", "Base SAM-2.1"], grid_size=(1, 2))